In [1]:
import numpy as np
import os
import math
#import _pickle as pickle
import imageio
import matplotlib.pyplot
import glob
import cv2
from scipy import ndimage
from PIL import Image, ImageFilter
%matplotlib inline

In [2]:
# format dataset from https://www.kaggle.com/oddrationale/mnist-in-csv
with open("mnist_dataset/mnist_train.csv", 'r') as f:
    with open("mnist_dataset/updated_train.csv", "w") as f1:
        next(f)
        for line in f:
            f1.write(line)
            
with open("mnist_dataset/mnist_test.csv", 'r') as f:
    with open("mnist_dataset/updated_test.csv", "w") as f1:
        next(f)
        for line in f:
            f1.write(line)
    
# read mnist dataset
training_data_file = open("mnist_dataset/updated_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

test_data_file = open("mnist_dataset/updated_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [3]:
# neural network class based on makeyourownneuralnetwork book
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        """
        initializer of neuralNetwork 
        inputnodes: number of nodes in input layer
        hiddennodes: number of nodes in hidden layer
        outputnodes: number of nodes in output layer
        learningrate: learning rate of neural network
        """
        self.nodes_in, self.nodes_hidden, self.nodes_out, self.rate_learning = inputnodes, hiddennodes, outputnodes, learningrate
        self.weights_hiddenout = np.random.normal(0, (self.nodes_hidden)**(-1/2), (self.nodes_out, self.nodes_hidden))
        self.weights_inhidden = np.random.normal(0, (self.nodes_in)**(-1/2), (self.nodes_hidden, self.nodes_in))
    
    def activation_function(self, x):
        def sigmoid(m):
            # numpy implementation of sigmoid function
            return 1.0/(1.0+np.exp(-m))
        return sigmoid(x)
        
    def train(self, inputs, targets):
        """
        train the neuralNetwork
        inputs: image value list
        targets: list of target values
        """
        i = np.array(inputs, ndmin=2).T
        t = np.array(targets, ndmin=2).T
        
        o_hidden = self.activation_function(np.dot(self.weights_inhidden, i))
        o_out = self.activation_function(np.dot(self.weights_hiddenout, o_hidden))
        err_out = t - o_out
        err_hidden = np.dot(self.weights_hiddenout.T, err_out)
        temp1, temp2 = o_out * (1.0-o_out) * err_out, o_hidden * (1.0-o_hidden) * err_hidden
        self.weights_inhidden += np.dot(temp2, np.transpose(i)) * (self.rate_learning)
        self.weights_hiddenout += np.dot(temp1, np.transpose(o_hidden)) * (self.rate_learning)
        
    def query(self, inputs):
        """
        query the neuralNetwork
        li: list of input values if uploadedImage is False else image value list
        uploadedImage: True if querying uploaded image
        """
        i = np.array(inputs, ndmin=2).T
        o_hidden = self.activation_function(np.dot(self.weights_inhidden, i))
        o_out = self.activation_function(np.dot(self.weights_hiddenout, o_hidden))
        return o_out
    
    # save neural network weights
    def save(self):
        np.save('digit_wih.npy', self.weights_inhidden)
        np.save('digit_who.npy', self.weights_hiddenout)

    # load neural network weights
    def load(self):
        self.weights_inhidden = np.load('digit_wih.npy')
        self.weights_hiddenout = np.load('digit_who.npy')


In [4]:
def getInputs(arr):
    # function returns list of input values from image value list
    return 0.01 + (np.asfarray(arr[1:]) * float(99/100) * float(1/255))



In [5]:
input_nodes, hidden_nodes, output_nodes, learning_rate = 784, 200, 10, 0.1
nn = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [7]:
# train the neural network 

epochs = 3

for _ in range(epochs):
    for image in training_data_list:
        vals = image.split(',')
        inputs = getInputs(vals)
        targets = np.zeros(output_nodes) + 0.01
        targets[int(vals[0])] = 0.99
        nn.train(inputs, targets)
        
        
        #inputs_rotright = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 12, cval=0.01, order=1, reshape=False)
        #nn.train(inputs_rotright.reshape(784), targets)
        #inputs_rotleft = scipy.ndimage.interpolation.rotate(inputs.reshape(28, 28), -12, cval=0.01, order=1, reshape=False)
        #nn.train(inputs_rotleft.reshape(784), targets)
        
nn.save()

In [8]:
# test the neural network
tlist = []

for image in test_data_list:
    vals = image.split(',')
    correct_val = int(vals[0])
    inputs = getInputs(vals)
    outputs = nn.query(inputs)
    output_val = np.argmax(outputs)
    tlist.append(int(output_val==correct_val))
    
# calculate accuracy performance of neural network
test_result = np.asarray(tlist)
acc_performance = test_result.sum()/test_result.size
print("Accuracy Performance: ", acc_performance)

Accuracy Performance:  0.9723


In [9]:
def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

In [47]:
def imageProcessing(filepath):
    imagedata = []
    for img in glob.glob(filepath):
        label = int(img[-5:-4])
        edit_image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
        edit_image = cv2.resize(255-edit_image, (28, 28))
        (thresh, edit_image) = cv2.threshold(edit_image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        while np.sum(edit_image[0]) == 0:
            edit_image = edit_image[1:]

        while np.sum(edit_image[:,0]) == 0:
            edit_image = np.delete(edit_image,0,1)

        while np.sum(edit_image[-1]) == 0:
            edit_image = edit_image[:-1]

        while np.sum(edit_image[:,-1]) == 0:
            edit_image = np.delete(edit_image,-1,1)

        rows,cols = edit_image.shape
        if rows > cols:
            factor = 20.0/rows
            rows = 20
            cols = int(round(cols*factor))
            edit_image = cv2.resize(edit_image, (cols,rows))
        else:
            factor = 20.0/cols
            cols = 20
            rows = int(round(rows*factor))
            edit_image = cv2.resize(edit_image, (cols, rows))

        colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
        rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
        edit_image = np.lib.pad(edit_image,(rowsPadding,colsPadding),'constant')
        shiftx,shifty = getBestShift(edit_image)
        shifted = shift(edit_image,shiftx,shifty)
        edit_image = shifted
        edit_image = edit_image.flatten()
        record = np.append(label,edit_image)
        imagedata.append(record)
    return imagedata


In [48]:
imagedata = imageProcessing('static/img_test_?.png')
for i in range(len(imagedata)):
    label = imagedata[i][0]
    inputs = imagedata[i][1:]
    outputs = nn.query(inputs)
    returnval = np.argmax(outputs)
    print("network says: " + str(returnval))
    if returnval == label:
        print("correct!")
    else:
        print("incorrect!")

network says: 0
correct!
network says: 1
correct!
network says: 2
correct!
network says: 3
correct!
network says: 4
correct!
network says: 5
correct!
network says: 6
correct!
network says: 7
correct!
network says: 8
correct!
network says: 9
correct!


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in exp
